In [1]:
#Load in utility classes
%run ~/projects/testlipids/testlipids.ipynb
import os #Operating system specific commands
import re #Regular expression library

membrane="epithelial"
insane="./insane+SF.py"
mdparams="./test.mdp"
martinipath="./martini.ff"

In [1]:
# Cleaning up intermediate files from previous runs
!rm -f *#*
!rm -f *step*
!rm -f {membrane}*

In [111]:
upper="-u LPPC:0.02 -u DPPC:0.007 -u DOPC:0.026 -u PIPC:0.217 -u PAPC:0.128 -u PUPC:0.123 -u POPE:0.017 -u PUPE:0.005 -u DPSM:0.043 -u PNSM:0.019"
#-u POPC:0.363 -u PAPE:0.014 -u DOPE:0.018 
lower= "-l LPPC:0.007 -l DPPC:0.003 -l DOPC:0.009 -l PIPC:0.078 -l PAPC:0.046 -l PUPC:0.044 -l POPA:0.022 -l DOPA:0.011 -l POPE:0.057 -l PUPE:0.017 -l DPPS:0.002 -l POPS:0.216 -l POPI:0.058 -l PIPI:0.013 -l PAPI:0.128 -l PUPI:0.018 -l DPSM:0.015 -l PNSM:0.007"
# -l POPC:0.129 -l DOPE:0.06 -l PAPE:0.047 -l DOPS:0.014

In [112]:
#upper="-u LPPC:0.02 -u DPPC:0.007 -u POPC:0.363 -u DOPC:0.026 -u PIPC:0.217 -u PAPC:0.128 -u PUPC:0.123 -u POPE:0.017 -u DOPE:0.018 -u PAPE:0.014 -u PUPE:0.005 -u DPSM:0.043 -u PNSM:0.019 "
#lower= "-l LPPC:0.007 -l DPPC:0.003 -l POPC:0.129 -l DOPC:0.009 -l PIPC:0.078 -l PAPC:0.046 -l PUPC:0.044 -l POPA:0.022 -l DOPA:0.011 -l POPE:0.057 -l DOPE:0.06 -l PAPE:0.047 -l PUPE:0.017 -l DPPS:0.002 -l POPS:0.216 -l DOPS:0.014 -l POPI:0.058 -l PIPI:0.013 -l PAPI:0.128 -l PUPI:0.018 -l DPSM:0.015 -l PNSM:0.007"


print("Build")
build = !python2 {insane} -o {membrane}.gro -p {membrane}.top -d 0 -x 16.6 -y 19.4 -z 16 -sol PW -salt 0.15 -center -charge 0 -orient {upper} {lower}  
#for line in build:
#    print(line)

print("Grompp")
grompp = !gmx grompp -f {mdparams} -c {membrane}.gro -p {membrane}.top -o  {membrane}.tpr
success=True
for line in grompp:
    if re.search("Fatal error", line):
        success=False
    if not success:
        print(line)
    
if success:
    print("Run")
    !export GMX_MAXCONSTRWARN=-1
    !export GMX_SUPPRESS_DUMP=1
    run = !gmx mdrun -v -deffnm {membrane}
    summary=""
    logfile = membrane+".log"
    if not os.path.exists(logfile):
        print("no log file")
        print("== === ====")
        for line in run:
            print(line)
    else:
        try:
            file = open(logfile, "r")
            fe = False
            for line in file:
                if fe:
                    success=False
                    summary=line
                elif re.search("^Steepest Descents.*converge", line):
                    success=True
                    summary=line
                    break
                elif re.search("Fatal error", line):
                    fe = True
        except IOError as exc:
            sucess=False;
            summary=exc;
    if success:
        print("Success")
    else:
        print(summary)


Build
Grompp
Run
Success
